# 

In [1]:
import pandas as pd
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from torch.utils.data import TensorDataset

from sklearn.model_selection import train_test_split

from data_processing.encoding.categorical import CategoricalTransformer
from data_processing.encoding.numeric_and_date import ToQuantileTransformer
from data_processing.encoding.text2vec import TextTransform
from data_processing.sequences.sequencing import to_sequences

In [2]:
# read in data and prepare transformations
data = pd.read_csv('data/other_data/PreFer_fake_data.csv')
targets = pd.read_csv('data/other_data/PreFer_fake_outcome.csv')
codebook = pd.read_csv('data/codebooks/PreFer_codebook.csv')
summary = pd.read_csv('data/codebooks/PreFer_codebook_summary.csv')

categorical_columns = codebook[(codebook.var_name.str.startswith('c')) & (codebook.type_var == 'categorical')].var_name.tolist()
quantile_columns = codebook[(codebook.var_name.str.startswith('c')) & ((codebook.type_var == 'numeric') | (codebook.type_var == 'date or time'))].var_name.tolist()

cat_transform = CategoricalTransformer()
cat_transform.fit(codebook)

/Users/lmmi/fertility-prediction-challenge/data_processing/encoding/categorical.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  core_cat_df['values_cat'] = core_cat_df['values_cat'].str.split("; ").apply(lambda x: [e.strip() for e in x])
/Users/lmmi/fertility-prediction-challenge/data_processing/encoding/categorical.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  core_cat_df['labels_cat'] = core_cat_df['labels_cat'].str.split("; ").apply(lambda x: [e.strip() for e in x])


In [3]:
# tokenize and store as sequences
for col in categorical_columns:
    data[col] = cat_transform.transform(data[col])
    
quantile_transform = ToQuantileTransformer(quantile_columns)
quantile_transform.fit(data)
data = quantile_transform.transform(data)

data.fillna(101, inplace=True)
data[quantile_columns] = data[quantile_columns].astype(int)

data2 = data[data.columns[data.columns.str.startswith('c')]]
data2['nomem_encr'] = data['nomem_encr']

sequences = to_sequences(data2, summary)

/var/folders/y6/j9fbqcvx6lb5l99614n30y4c0000gn/T/ipykernel_6228/2027597144.py:13: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data2['nomem_encr'] = data['nomem_encr']
/var/folders/y6/j9fbqcvx6lb5l99614n30y4c0000gn/T/ipykernel_6228/2027597144.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data2['nomem_encr'] = data['nomem_encr']


In [4]:
def get_device():
    # Check if CUDA is available
    if torch.cuda.is_available():
        # If CUDA is available, select the first CUDA device
        device = torch.device("cuda:0")
        print("Using CUDA device:", torch.cuda.get_device_name(0))
    # Check for MPS availability on supported macOS devices (requires PyTorch 1.12 or newer)
    elif torch.backends.mps.is_available():
        # If MPS is available, use MPS device
        device = torch.device("mps")
        print("Using MPS (Metal Performance Shaders) device")
    else:
        # Fallback to CPU if neither CUDA nor MPS is available
        device = torch.device("cpu")
        print("Using CPU")
    return device
device = 'cpu' #get_device()

In [7]:
import numpy as np
index = np.array([isinstance(x, str) for x in sequences[700001]['08']])
np.array(sequences[700001]['08'])[index]

array(['long answer', 'answer', 'answ', 'answ', 'answer', 'answer',
       'answer', 'answ', 'long answer', 'answ', 'answer', 'answer',
       'answ', 'answer', 'answ', 'answer', 'answ', 'answ', 'answer',
       'answer', 'answ', 'answ', 'answer', 'answ', 'answ', 'answer',
       'even longer answer', 'answer', 'answer', 'answer', 'answ',
       'answer', 'answ', 'answer', 'answer', 'answer', 'long answer',
       'answ', 'answer', 'long answer', 'answ', 'answ', 'answ', 'answer',
       'even longer answer', 'answ', 'answer', 'answer', 'answer',
       'even longer answer', 'answer', 'answer', 'answ', 'answer', 'answ',
       'answer', 'answer', 'long answer', 'answer', 'answ',
       'even longer answer', 'answer', 'answ', 'answer', 'answer',
       'answer', 'long answer', 'answer', 'answ', 'answ', 'answ',
       'answer', 'even longer answer', 'answer', 'answer'], dtype='<U32')

In [5]:
# replace strings in sequences with 101 for nan
for _, wave_responses in sequences.items():
    for year, wave_response in wave_responses.items():
        
        not_int = np.array([not isinstance(x, int) for x in wave_response], dtype = bool)
        wave_responses[year] = [
            item if not _bool else 101 for (item, _bool) in zip(wave_response, not_int)
        ]

In [6]:
autoencoder_data = torch.tensor([
                                wave_response
                                for _, wave_responses in sequences.items()
                                for _, wave_response in wave_responses.items()
                        ]).to(torch.int64)

In [33]:
targets = pd.DataFrame(
    {'nomem_encr':np.arange(n_people),
     'target':np.random.randint(low = 0, high = 2, size = n_people)
    }
)

NameError: name 'n_people' is not defined

In [15]:
from model.autoencoder import AutoEncoder

In [170]:
class Autoencoder(nn.Module):
    """Makes the main denoising auto

    Parameters
    ----------
    in_shape [int] : input shape
    enc_shape [int] : desired encoded shape
    """

    def __init__(self, in_shape, enc_shape):
        super(Autoencoder, self).__init__()
        
        self.encode = nn.Sequential(
            nn.Linear(in_shape, 128),
            nn.ReLU(True),
            nn.Dropout(0.2),
            nn.Linear(128, 64),
            nn.ReLU(True),
            nn.Dropout(0.2),
            nn.Linear(64, enc_shape),
        )
        
        self.decode = nn.Sequential(
            nn.BatchNorm1d(enc_shape),
            nn.Linear(enc_shape, 64),
            nn.ReLU(True),
            nn.Dropout(0.2),
            nn.Linear(64, 128),
            nn.ReLU(True),
            nn.Dropout(0.2),
            nn.Linear(128, in_shape)
        )
        
    def forward(self, x):
        x = self.encode(x)
        x = self.decode(x)
        return x



In [8]:
autoencoder_data.shape

torch.Size([420, 3121])

In [12]:
autoencoder_data.dtype

torch.int32

In [16]:
batch_size = 128 

train_dataloader = DataLoader(autoencoder_data, batch_size=batch_size, shuffle=True)

num_epochs_autoencoder = 10
learning_rate_autoencoder = 0.001
embedding_dim = 128

n_questions = autoencoder_data.shape[1] 

error = nn.MSELoss()

autoencoder = AutoEncoder(num_embeddings=n_questions * embedding_dim).to(device)

optimizer = optim.Adam( autoencoder.parameters())

In [17]:
autoencoder.train()
for epoch in range(num_epochs_autoencoder):
    for batch in train_dataloader:
        optimizer.zero_grad()
        loss = autoencoder.get_loss(batch[0].to(device))
        
        loss.backward()
        
        optimizer.step()
        if epoch % int(0.01*num_epochs_autoencoder) == 0:
            print(f'epoch {epoch} \t Loss: {loss.item():.4g}')

epoch 0 	 Loss: 0.988
epoch 0 	 Loss: 0.9875
epoch 0 	 Loss: 0.9781
epoch 0 	 Loss: 0.9675
epoch 1 	 Loss: 0.9597
epoch 1 	 Loss: 0.9476
epoch 1 	 Loss: 0.8431
epoch 1 	 Loss: 0.932
epoch 2 	 Loss: 0.9188
epoch 2 	 Loss: 0.8882
epoch 2 	 Loss: 0.8687
epoch 2 	 Loss: 0.8422
epoch 3 	 Loss: 0.801
epoch 3 	 Loss: 0.5053
epoch 3 	 Loss: 0.7267
epoch 3 	 Loss: 0.6643
epoch 4 	 Loss: 0.6573
epoch 4 	 Loss: 0.6777
epoch 4 	 Loss: 0.7135
epoch 4 	 Loss: 0.661
epoch 5 	 Loss: 0.6305
epoch 5 	 Loss: 0.5739
epoch 5 	 Loss: 0.5615
epoch 5 	 Loss: 0.5967
epoch 6 	 Loss: 0.1169
epoch 6 	 Loss: 0.5343
epoch 6 	 Loss: 0.5719
epoch 6 	 Loss: 0.5871
epoch 7 	 Loss: 0.6107
epoch 7 	 Loss: 0.543
epoch 7 	 Loss: 0.5394
epoch 7 	 Loss: 0.5457
epoch 8 	 Loss: 0.5665
epoch 8 	 Loss: 0.5623
epoch 8 	 Loss: 0.5492
epoch 8 	 Loss: 0.5083
epoch 9 	 Loss: 0.555
epoch 9 	 Loss: 0.5297
epoch 9 	 Loss: 0.5317
epoch 9 	 Loss: 0.5136


In [118]:
targets.set_index(keys = 'nomem_encr').squeeze().to_dict()

{700001: 1,
 700002: 0,
 700003: 0,
 700004: 0,
 700005: 0,
 700006: 0,
 700007: 0,
 700008: 0,
 700009: 0,
 700010: 1,
 700011: 0,
 700012: 0,
 700013: 0,
 700014: 0,
 700015: 1,
 700016: 0,
 700017: 0,
 700018: 1,
 700019: 0,
 700020: 1,
 700021: 1,
 700022: 0,
 700023: 1,
 700024: 0,
 700025: 0,
 700026: 0,
 700027: 0,
 700028: 0,
 700029: 0,
 700030: 0}

In [119]:
class SequencesWithTarget(Dataset):
    def __init__(self, sequences:dict, target: pd.DataFrame):
        self.sequences = sequences 
        self.target = targets.set_index(keys = 'nomem_encr').squeeze().to_dict()
        self.keys = list(sequences.keys())

    def __len__(self):
        return len(self.keys)

    def __getitem__(self, index):
        person_id = self.keys[index]
        
        sequence = self.sequences[person_id]
        target = self.target[person_id]
        return target, sequence



In [120]:
train_person_ids, test_person_ids = train_test_split(targets['nomem_encr'], test_size=0.2, random_state=42)
train_person_ids, val_person_ids = train_test_split(train_person_ids, test_size=0.1, random_state=42)

train_data = {person_id: sequences[person_id] for person_id in train_person_ids}
val_data = {person_id: sequences[person_id] for person_id in val_person_ids}
test_data = {person_id: sequences[person_id] for person_id in test_person_ids}


In [124]:
train_dataset = SequencesWithTarget(train_data, target = targets)
val_dataset = SequencesWithTarget(val_data, target = targets)
test_dataset = SequencesWithTarget(test_data, target = targets)

rnn_batch_size = 10

train_dataloader = DataLoader(train_dataset, batch_size=rnn_batch_size, shuffle=True)
val_dataloader   = DataLoader(val_dataset,   batch_size=rnn_batch_size)
test_dataloader  = DataLoader(test_dataset,  batch_size=rnn_batch_size)

In [125]:
class CustomRNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers):
        super(CustomRNN, self).__init__()
        self.rnn = nn.RNN(input_size, hidden_size, num_layers=num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        output, _ = self.rnn(x)
        
        output_flat = output.contiguous().view(-1, output.size(2))
        
        logits = self.fc(output_flat)
        
        probabilities = self.sigmoid(logits)
        
        return probabilities.squeeze()

In [128]:
hidden_size = 10
n_layers = 3

# Initialize the RNN model
rnn_model = CustomRNN(
    input_size=n_questions,
    hidden_size=hidden_size,
    num_layers=n_layers,
)

num_epochs_rnn = 10
learning_rate_rnn = 0.001

# Define loss function and optimizer for RNN
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(rnn_model.parameters(), lr=learning_rate_rnn)

In [129]:
# Training loop
num_epochs = 1

rnn_model.train()  # Set the model to training mode

for epoch in range(num_epochs):
    
    running_loss = 0.0
    
    for batch in train_dataloader:
        inputs, labels = batch
        
        optimizer.zero_grad()  # Zero the parameter gradients
        
        # Forward pass
        outputs, _ = rnn_model(inputs)
        loss = criterion(outputs.squeeze(), labels)  # Assuming outputs are logits
        
        # Backward pass and optimization
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item() * inputs.size(0)
    
    # Calculate average loss for the epoch
    epoch_loss = running_loss / len(train_dataloader.dataset)
    
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}")

# After training, you can evaluate the model on the validation set and test set using similar loops


ValueError: RNN: Expected input to be 2D or 3D, got 1D tensor instead

In [ ]:

# Train the RNN model
for epoch in range(num_epochs_rnn):
    # Forward pass
    outputs = rnn_model(rnn_input)
    loss = criterion(outputs, rnn_targets)
    
    # Backward pass and optimization
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    print(f"Epoch [{epoch+1}/{num_epochs_rnn}] | Loss: {loss.item():.4f}")

